# 雀魂網頁相關(?)
處理 python <-> 網頁雀魂的各種雜事...

利用 selenium 開啟 Chrome 搭配 javascript 提供溝通的 interface

座標目前都用測試機量的絕對座標，理論上不同電腦會有偏差，
如果要量的話可以用下面這段程式碼抓 x,y 座標(但挺麻煩qwq)

In [ ]:
'''
from pynput import mouse, keyboard

def on_click(x, y, button, pressed):
    if pressed:
        print(f"pos: x={x}, y={y}")

def on_press(key):
    try:
        if key.char == 'e':
            print("Exit!")
            return False
    except AttributeError:
        pass

with mouse.Listener(on_click=on_click) as mouse_listener, keyboard.Listener(on_press=on_press) as keyboard_listener:
    print("Click to get mouse position, click 'e' to exit!")
    keyboard_listener.join() 
    mouse_listener.stop()
'''

### Import & Parameters

In [27]:
from selenium import webdriver
from pynput import mouse, keyboard
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyautogui
import time
import json
import sys
from enum import Enum, auto
class GameState(Enum):
    Loading = auto()
    Home = auto()
    InGame = auto()
OP_INTERVAL = 1 # click interval
current_state = None
current_hand = None
driver = None
auto_running = False
main_running = True

### Start Game & Auto Login

開啟一個 Chrome 瀏覽器連到雀魂並登入測試帳號

In [28]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
def start_game():
    global current_state, driver
    current_state = GameState.Loading
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get("https://game.maj-soul.com")
    driver.maximize_window()
    time.sleep(10)

    pyautogui.moveTo(1200, 430)
    pyautogui.click()
    # auto login: input account and password
    pyautogui.write("##YOUR ACCOUNT##", interval=0.05)
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1200, 520)
    pyautogui.click()
    pyautogui.write("##YOUR PASSWORD##", interval=0.05)
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1200, 670)
    pyautogui.click()

    # wait for the page to load
    time.sleep(12)

    # close the pop-up
    pyautogui.moveTo(1640, 370)
    pyautogui.click()
    time.sleep(OP_INTERVAL)
    current_state = GameState.Home
    print("Game started")

### Close browser

In [29]:
def close_browser():
    global driver
    driver.quit()
    driver = None
    print("Browser closed.")

### Create Bot Game

In [30]:
def enter_bot_game():
    global current_state
    if current_state != GameState.Home:
        print("Not in home state, Abort")
        return
    # create bot room
    pyautogui.moveTo(1280, 750)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1280, 500)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(965, 920)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    # add 3 bots
    pyautogui.moveTo(700, 670)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1000, 670)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(1300, 670)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    # start the game
    pyautogui.moveTo(1080, 950)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    # wait for the game to start
    # time.sleep(15)
    current_state = GameState.InGame
    print("Bot game started")

### Leave game

In [31]:
def leave_bot_game():
    global current_state
    if current_state != GameState.InGame:
        print("Not in game state, Abort")
        return
    
    pyautogui.moveTo(1645, 235)
    pyautogui.click()
    time.sleep(OP_INTERVAL)

    pyautogui.moveTo(825, 710)
    pyautogui.click()
    time.sleep(OP_INTERVAL)
    
    # Return to the main page time
    # time.sleep(5)
    current_state = GameState.Home
    print("Bot game left")

### 嵌入 js code

將我們的 web.js 嵌入，讓 python 可以透過 js 和網頁互動

In [32]:
def inject_js():
    global driver
    with open("web.js", "r", encoding="utf-8") as js_file:
        js_code = js_file.read()
    
    driver.execute_script(js_code)
    print("JS injected")

### Update & Visualize 手牌

將從雀魂網頁收到的手牌資訊 decode 整理成方便 model 使用的格式，
並提供易讀的 debug 訊息

In [33]:
def filter_hand_data(data):
    filtered_data = {'dora': data['dora'], 'type': data['type'], 'index': data['index']}
    return filtered_data
def update_hand_data(new_data):
    global current_hand
    filtered_data = [filter_hand_data(tile) for tile in new_data]
    current_hand = filtered_data

def visualize_tile(tile):
    dora, tile_type, index = tile['dora'], tile['type'], tile['index']
    if tile_type == 0:
        tile_type = '筒'
    elif tile_type == 1:
        tile_type = '萬'
    elif tile_type == 2:
        tile_type = '條'
    else:
        tile_type = '字'
    return f"{index} {tile_type} {'Dora' if dora else ''}"

def print_current_hand():
    global current_hand
    for i, tile in enumerate(current_hand):
        print(f"tile {i}: {visualize_tile(tile)}")

### Press Event


In [34]:
def on_press(key):
    global auto_running, main_running, driver
    hand_data = None
    try:
        '''
        if key.char == 'e':
            print("'e' key pressed, leaving bot game.")
            leave_bot_game()
        elif key.char == 's':
            print("'s' key pressed, starting bot game.")
            enter_bot_game()
        elif key.char == 'q':
            print("'q' key pressed, closing browser.")
            close_browser()
            main_running = False
            sys.exit(0)
        elif key.char == 'h':
            print("'h' key pressed, Helloworld.")
            driver.execute_script("helloword();")
        elif key.char == 'g':
            print("'g' key pressed, get Hand Data.")
            hand_data = driver.execute_script("return GetHandData();")
            if hand_data is None:
                print("手牌資料未返回，請檢查 JavaScript 函數的返回值！")
            else:
                update_hand_data(hand_data)
        elif key.char == 'p':
            print("'p' key pressed, print hand data.")
            print_current_hand()
        ###    
        elif key.char == 'd':
            print("'d' key pressed, discard tile.")
            # before discard
            # hand_data = driver.execute_script("return GetHandData();")
            # update_hand_data(hand_data)
            # print("Before discard:")
            # print_current_hand()
            result = driver.execute_async_script("""
                const done = arguments[arguments.length - 1];

                discard(window.ownHand)
                    .then(tile => done(tile))    
                    .catch(err => done(err.message));
            """)
            print(f"丟出的牌: {visualize_tile(filter_hand_data(result))}")
        '''
        if key.char == 't':
            print("'t' key pressed, Toggle Auto Play.")
            driver.execute_script("window.Auto_run = !window.Auto_run;")
            auto_running = not auto_running
    except AttributeError:
        pass

## Import Model

In [35]:
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow
import numpy as np
from chi_pon_kong_RL import ChiPongKongEnvironment
from chi_pon_kong_RL import RLAgent
from chi_pon_kong_RL import train_agent

rl_env = ChiPongKongEnvironment()
rl_agent = RLAgent(action_space=["chi", "pong", "kong", "skip"])
#train_agent()

class Model_NN():
    def __init__(self) -> None:
        self.model = Sequential([Dense(64, activation='relu', input_shape=(34,)),
                                 Dense(128, activation='relu'),
                                 Dense(256, activation='relu'),
                                 Dropout(0.2),
                                 Dense(34, activation='softmax')
                                ])
        self.model.compile(optimizer= tensorflow.keras.optimizers.Adam(learning_rate=1e-3),loss='categorical_crossentropy', metrics=['accuracy'])       
        self.model.load_weights('checkpoints\model_epoch_10.h5')

    def filter_prediction(self, input_tiles, prediction):
        """
        filter the prediction to make sure the output is a legal tile index
        
        Args:
            input_tiles (np.array): player's hand (length = 34 vec, one-hot encoding)
            prediction (np.array): model's prediction (length = 34 vec, softmax output)
        
        Returns:
            int: filtered prediction index
        """
        legal_tiles = np.where(input_tiles > 0)[1]
        prediction = prediction.reshape(34,)
        filtered_prediction = np.zeros_like(prediction)
        
        
        filtered_prediction[legal_tiles] = prediction[legal_tiles]
        
        return np.argmax(filtered_prediction)

    def discard(self, hands_input):
        hands = np.zeros(34) 

        for tile in hands_input:
            hands[tile['type'] * 9 + tile['index'] - 1] += 1

        hands = hands.reshape(1, 34) 
        out = self.filter_prediction(hands, self.model.predict(hands))

        for i, tile in enumerate(hands_input):
            if tile['type'] * 9 + tile['index'] - 1 == out:
                return i
            
model = Model_NN()

<>:21: SyntaxWarning: invalid escape sequence '\m'
<>:21: SyntaxWarning: invalid escape sequence '\m'
C:\Users\danny\AppData\Local\Temp\ipykernel_25892\3438426243.py:21: SyntaxWarning: invalid escape sequence '\m'
  self.model.load_weights('checkpoints\model_epoch_10.h5')
c:\Users\danny\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Main Loop to start Game

啟動遊戲的主迴圈，會自動偵測當前狀態，若輪到自己則 call model 去取得要做的選擇 

In [36]:

def main():
    global auto_running, driver, main_running
    start_game()
    inject_js()
    with keyboard.Listener(on_press=on_press) as keyboard_listener:
        print(main_running)
        while main_running:
            if not auto_running:
                print("Auto_run is False")
                time.sleep(0.5)
                continue
            # get operations list
            operations = driver.execute_script("return getOperationList(1);")
            operations = json.loads(operations)
            print(f"Main Loop {operations}")
            
            if not operations or len(operations) == 0:
                print("Waiting for own turn.")
                time.sleep(0.5)
                continue
            
            main_own_turn()
            time.sleep(1)
def main_own_turn():
    global driver, current_hand, rl_agent, rl_env 
    if driver.execute_script("return window.threadIsRunning;"):
        return
    driver.execute_script("window.threadIsRunning = true;")
    driver.execute_script("GetHandData();")
    
    operations = driver.execute_script("return getOperationList(1);")
    operations = json.loads(operations)
    print("##### OWN TURN #####")
    
    for operation in operations:
        if driver.execute_script("return getOperationList(1).length;") == 0:
            break
        operation_type = operation.get('type')
        
        prev_state = rl_env.state.copy() 
        
        if operation_type == driver.execute_script("return getOperations().eat;"): 
            action = rl_agent.choose_action(rl_env.state)   
            if action == "chi":
                driver.execute_script(f"callEat({1});")
                next_state, reward, done = rl_env.step("chi")   
            else:
                driver.execute_script(f"callEat({0});")
                next_state, reward, done = rl_env.step("skip")
            
            # RL learn
            rl_agent.learn(prev_state, action, reward, next_state)
            
        elif operation_type == driver.execute_script("return getOperations().peng;"):
            # RL decide whether to execute pong
            action = rl_agent.choose_action(rl_env.state)  # Get RL action
            if action == "pong":
                driver.execute_script(f"callPeng({1});")
                next_state, reward, done = rl_env.step("pong")  
            else:
                driver.execute_script(f"callPeng({0});")
                next_state, reward, done = rl_env.step("skip")
            
            # RL learn
            rl_agent.learn(prev_state, action, reward, next_state)
            
        elif operation_type == driver.execute_script("return getOperations().ming_gang;"):
            # RL decide whether to execute kong
            action = rl_agent.choose_action(rl_env.state)  # Get RL action
            if action == "kong":
                driver.execute_script(f"callDaiminkan({1});")
                next_state, reward, done = rl_env.step("kong")  
            else:
                driver.execute_script(f"callDaiminkan({0});")
                next_state, reward, done = rl_env.step("skip")
            
            # RL learn
            rl_agent.learn(prev_state, action, reward, next_state)
    
    print("Own turn completed.")
    driver.execute_script("window.threadIsRunning = false;")

def main_own_turn():
    global driver, current_hand, rl_agent, rl_env 
    if driver.execute_script("return window.threadIsRunning;"):
        return
    driver.execute_script("window.threadIsRunning = true;")
    driver.execute_script("GetHandData();")
    
    operations = driver.execute_script("return getOperationList(1);")
    operations = json.loads(operations)
    print("##### OWN TURN #####")
    
    for operation in operations:
        if driver.execute_script("return getOperationList(1).length;") == 0:
            break
        operation_type = operation.get('type')
        prev_state = rl_env.state.copy() 
        if operation_type == driver.execute_script("return getOperations().an_gang;"):
            driver.execute_script(f"callAnkan({0}, {operation['combination']});")
        elif operation_type == driver.execute_script("return getOperations().add_gang;"):
            driver.execute_script(f"callShouminkan({0});")
        elif operation_type == driver.execute_script("return getOperations().zimo;"):
            driver.execute_script("callTsumo();")
        elif operation_type == driver.execute_script("return getOperations().rong;"):
            driver.execute_script("callRon();")
        elif operation_type == driver.execute_script("return getOperations().jiuzhongjiupai;"):
            driver.execute_script(f"callAbortiveDraw({0});")
    
    for operation in operations:
        if driver.execute_script("return getOperationList(1).length;") == 0:
            break
        operation_type = operation.get('type')
        
        if operation_type == driver.execute_script("return getOperations().dapai;"):
            hand_data = driver.execute_script("return GetHandData();")
            update_hand_data(hand_data)
            richi = False
            comb = None
            for op in operations:
                if op['type'] == driver.execute_script("return getOperations().liqi;"):
                    richi = True
                    comb = op['combination']
                    break
            if richi:
                # Todo: ask richi model, for now just call richi
                print("Richi!"+str(comb))
                for c in comb:
                    print(f"Combination: {c}")
                    js_argument = json.dumps(c)
                    driver.execute_script(f"callRiichi({js_argument});")
                    break
            else:
                tile_idx = model.discard(current_hand)
                driver.execute_script(f"discard_idx({tile_idx});") 
                # driver.execute_script("discard(window.ownHand);")
        elif operation_type == driver.execute_script("return getOperations().eat;"):
            action = rl_agent.choose_action(rl_env.state) 
            if action == "chi":
                driver.execute_script(f"callEat({1});")
                next_state, reward, done = rl_env.step("chi") 
            else:
                driver.execute_script(f"callEat({0});")
                next_state, reward, done = rl_env.step("skip")
            
            # RL 学习
            rl_agent.learn(prev_state, action, reward, next_state)
        elif operation_type == driver.execute_script("return getOperations().peng;"):
            action = rl_agent.choose_action(rl_env.state)
            if action == "pong":
                driver.execute_script(f"callPeng({1});")
                next_state, reward, done = rl_env.step("pong") 
            else:
                driver.execute_script(f"callPeng({0});")
                next_state, reward, done = rl_env.step("skip")
            
            rl_agent.learn(prev_state, action, reward, next_state)
        elif operation_type == driver.execute_script("return getOperations().ming_gang;"):
            action = rl_agent.choose_action(rl_env.state)  
            if action == "kong":
                driver.execute_script(f"callDaiminkan({1});")
                next_state, reward, done = rl_env.step("kong") 
            else:
                driver.execute_script(f"callDaiminkan({0});")
                next_state, reward, done = rl_env.step("skip")
            
            rl_agent.learn(prev_state, action, reward, next_state)
    rl_agent.save_q_table("q_table.pkl")
    print("Q-Table saved.")
    print("Own turn completed.")
    driver.execute_script("window.threadIsRunning = false;")


In [ ]:
main()

Game started
JavaScript 已成功嵌入！
True
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
Auto_run is False
